# 3rd part cluster neighborhood toronto

## import the libraries necessary

In [20]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

import json

# import k-means from clustering stage
from sklearn.cluster import KMeans

!pip install geopy
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


read both de csv file and drop the borough not assigned and then join them together with inner-left 

In [2]:
df = pd.read_csv('data_toronto.csv')
df=df[df['Borough']!='Not assigned']

df=df.set_index("PostalCode")
#df=df.reset_index()


df_pos=pd.read_csv('Geospatial_Coordinates.csv')
df_pos=df_pos.set_index("Postal Code")
df_pos.head()
df=df.join(df_pos)

df.head()

,Borough,Neighborhood,Latitude,Longitude
PostalCode,,,,
M3A,North York,Parkwoods,43.753259,-79.329656
M4A,North York,Victoria Village,43.725882,-79.315572
M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


get only the borough with the part "Toronto"

In [3]:
df=df.loc[df["Borough"].str.contains("Toronto")]
df=df.reset_index()
df.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
4,M4E,East Toronto,The Beaches,43.676357,-79.293031


#### Define Foursquare Credentials and Version

In [4]:
CLIENT_ID = 'SP12SEIU21VTIVFBT24UAKQMHPMLHPVL2BSJT0NUWQ0LW104' # your Foursquare ID
CLIENT_SECRET = 'S2XP3TRGW5DDTDDVIAMYDM3EV4DLRMDRUZ2QKWAPDNVT3S45' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: SP12SEIU21VTIVFBT24UAKQMHPMLHPVL2BSJT0NUWQ0LW104
CLIENT_SECRET:S2XP3TRGW5DDTDDVIAMYDM3EV4DLRMDRUZ2QKWAPDNVT3S45


def a function to get the venues for a single place (double-clic for more)
<!--
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

radius=500
LIMIT=100

latitude = df.loc[0, 'Latitude']
longitude= df.loc[0, 'Longitude']

url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, radius, LIMIT)

results = requests.get(url).json()["response"]['groups'][0]['items']
nearby_venues = json_normalize(results)

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()
-->         

In [5]:
radius=500
LIMIT=100

def getNearbyVenues(PostalCode, latitudes, longitudes, radius=500, LIMIT=100):
    
    venues_list=[]
    for PostalCode, lat, lng in zip(PostalCode, latitudes, longitudes):
        print(PostalCode)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        # return only relevant information for each nearby venue
        for v in results:
            venues_list.append([(
                PostalCode, 
                lat, 
                lng, 
                v['venue']['name'], 
                v['venue']['location']['lat'], 
                v['venue']['location']['lng'],  
                v['venue']['categories'][0]['name'])])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['PostalCode', 
                  'Latitude', 
                  'Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

try the function with a single position (double-clic)
<!--
PostalCode = df.loc[0, 'PostalCode']
lat = df.loc[0, 'Latitude']
lng = df.loc[0, 'Longitude']
print(PostalCode)
print(latitude)
print(longitude)
venues_list=[]
--> 

<!--
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
print(url)
--> 

<!--
results = requests.get(url).json()["response"]['groups'][0]['items']
# results = json_normalize(results)
# print(results)
for v in results:
#     print(v['venue']['categories'][0]['name'])        
    venues_list.append([(
                PostalCode, 
                lat, 
                lng, 
                v['venue']['name'], 
                v['venue']['location']['lat'], 
                v['venue']['location']['lng'],  
                v['venue']['categories'][0]['name'])])
-->     
<!--    
nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
nearby_venues
# venues_list.append([(
#             name, 
#             lat, 
#             lng, 
#             v['venue']['name'], 
#             v['venue']['location']['lat'], 
#             v['venue']['location']['lng'],  
#             v['venue']['categories'][0]['name']) for v in results])
--> 


### get the venus with the data frame

In [6]:
PostalCode=df['PostalCode']
latitudes=df['Latitude']
longitudes=df['Longitude']

In [7]:
Toronto_venues = getNearbyVenues(PostalCode,latitudes,longitudes)

M5A
M7A
M5B
M5C
M4E
M5E
M5G
M6G
M5H
M6H
M5J
M6J
M4K
M5K
M6K
M4L
M5L
M4M
M4N
M5N
M4P
M5P
M6P
M4R
M5R
M6R
M4S
M5S
M6S
M4T
M5T
M4V
M5V
M4W
M5W
M4X
M5X
M4Y
M7Y


In [8]:
Toronto_venues.shape

(1630, 7)

In [9]:
Toronto_venues.groupby('PostalCode').count()

,Latitude,Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
PostalCode,,,,,,
M4E,5,5,5,5,5,5
M4K,42,42,42,42,42,42
M4L,19,19,19,19,19,19
M4M,41,41,41,41,41,41
M4N,3,3,3,3,3,3
M4P,7,7,7,7,7,7
M4R,18,18,18,18,18,18
M4S,34,34,34,34,34,34
M4T,2,2,2,2,2,2


In [10]:
print('There are {} uniques categories.'.format(len(Toronto_venues['Venue Category'].unique())))

There are 229 uniques categories.


### create the one hot data frame

In [11]:
# one hot encoding
Toronto_onehot = pd.get_dummies(Toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
Toronto_onehot['PostalCode'] = Toronto_venues['PostalCode'] 

# move neighborhood column to the first column
fixed_columns = [Toronto_onehot.columns[-1]] + list(Toronto_onehot.columns[:-1])
Toronto_onehot = Toronto_onehot[fixed_columns]

Toronto_onehot.shape

(1630, 230)

In [12]:
Toronto_grouped = Toronto_onehot.groupby('PostalCode').mean().reset_index()
Toronto_grouped.shape

(39, 230)

### find the most frequence venues in each postcode

In [13]:
num_top_venues = 5

for PostalCode in Toronto_grouped['PostalCode']:
    print("----"+PostalCode+"----")
    temp = Toronto_grouped[Toronto_grouped['PostalCode'] == PostalCode].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----M4E----
               venue  freq
0                Pub   0.2
1       Neighborhood   0.2
2  Health Food Store   0.2
3   Asian Restaurant   0.2
4              Trail   0.2


----M4K----
                    venue  freq
0        Greek Restaurant  0.19
1             Coffee Shop  0.07
2      Italian Restaurant  0.07
3              Restaurant  0.05
4  Furniture / Home Store  0.05


----M4L----
                  venue  freq
0                  Park  0.11
1     Fish & Chips Shop  0.05
2          Intersection  0.05
3  Fast Food Restaurant  0.05
4               Brewery  0.05


----M4M----
         venue  freq
0         Café  0.10
1       Bakery  0.07
2  Coffee Shop  0.07
3    Gastropub  0.05
4      Brewery  0.05


----M4N----
               venue  freq
0               Park  0.33
1           Bus Line  0.33
2        Swim School  0.33
3  Afghan Restaurant  0.00
4      Movie Theater  0.00


----M4P----
               venue  freq
0              Hotel  0.29
1     Breakfast Spot  0.14
2     Sandwich 

## create the data frame shows the 10 most popular venues

In [14]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [15]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['PostalCode']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
PostalCode_venues_sorted = pd.DataFrame(columns=columns)
PostalCode_venues_sorted['PostalCode'] = Toronto_grouped['PostalCode']

for ind in np.arange(Toronto_grouped.shape[0]):
    PostalCode_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Toronto_grouped.iloc[ind, :], num_top_venues)

PostalCode_venues_sorted.head()

,PostalCode,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,Asian Restaurant,Trail,Neighborhood,Pub,Health Food Store,Donut Shop,Doner Restaurant,Dumpling Restaurant,Deli / Bodega,Dog Run
1,M4K,Greek Restaurant,Italian Restaurant,Coffee Shop,Restaurant,Ice Cream Shop,Furniture / Home Store,Fruit & Vegetable Store,Pub,Pizza Place,Lounge
2,M4L,Park,Pub,Liquor Store,Burrito Place,Sandwich Place,Italian Restaurant,Restaurant,Intersection,Fast Food Restaurant,Steakhouse
3,M4M,Café,Bakery,Coffee Shop,Brewery,Gastropub,American Restaurant,Neighborhood,Seafood Restaurant,Sandwich Place,Cheese Shop
4,M4N,Park,Bus Line,Swim School,Department Store,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run


## cluster the postalcode with K means method, we use a k of 5

In [16]:
# set number of clusters
kclusters = 5

Toronto_grouped_clustering = Toronto_grouped.drop('PostalCode', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 2, 2, 2, 0, 0, 2, 2, 1, 2], dtype=int32)

In [17]:

PostalCode_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

Toronto_merged = df

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
Toronto_merged = Toronto_merged.join(PostalCode_venues_sorted.set_index('PostalCode'), on='PostalCode')

Toronto_merged.head() # check the last columns!

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,2,Coffee Shop,Park,Bakery,Pub,Breakfast Spot,Café,Theater,Mexican Restaurant,Shoe Store,Restaurant
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,2,Coffee Shop,Diner,Park,Bar,Beer Bar,Smoothie Shop,Sandwich Place,Burrito Place,Restaurant,Café
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,2,Coffee Shop,Clothing Store,Italian Restaurant,Café,Bubble Tea Shop,Cosmetics Shop,Japanese Restaurant,Diner,Pizza Place,Hotel
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,2,Café,Coffee Shop,Restaurant,Clothing Store,Cocktail Bar,Cosmetics Shop,American Restaurant,Creperie,Italian Restaurant,Beer Bar
4,M4E,East Toronto,The Beaches,43.676357,-79.293031,0,Asian Restaurant,Trail,Neighborhood,Pub,Health Food Store,Donut Shop,Doner Restaurant,Dumpling Restaurant,Deli / Bodega,Dog Run


## show the results in the map

In [21]:
# create map
address = 'Toronto, CA'

geolocator = Nominatim(user_agent="To_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Toronto_merged['Latitude'], Toronto_merged['Longitude'], Toronto_merged['Neighborhood'], Toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters